In [3]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
from langchain import PromptTemplate

prompt_template = """
   "Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!
   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,
   or anything else related to AIT, I'm here to help. Just ask away, and I'll provide you with all the necessary details
   to make your experience at AIT as smooth and enjoyable as possible."
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template='"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!\n   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,\n   or anything else related to AIT, I\'m here to help. Just ask away, and I\'ll provide you with all the necessary details\n   to make your experience at AIT as smooth and enjoyable as possible."\n    {context}\n    Question: {question}\n    Answer:')

In [6]:
PROMPT.format(
    context = "AIT is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.",
    question = "Tell me about AIT"
)

'"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!\n   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,\n   or anything else related to AIT, I\'m here to help. Just ask away, and I\'ll provide you with all the necessary details\n   to make your experience at AIT as smooth and enjoyable as possible."\n    AIT is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.\n    Question: Tell me about AIT\n    Answer:'

### Retrieval

##### Document Loaders

In [7]:
# cd content

In [8]:
from langchain.document_loaders import PyMuPDFLoader, TextLoader

nlp_docs = './data/AIT-BROCHURE.pdf'


pdf_loader1 = PyMuPDFLoader(nlp_docs)
document1 = pdf_loader1.load()

pdf_loader2 = PyMuPDFLoader('./data/090405.pdf')
document2 = pdf_loader2.load()

text_loader = TextLoader('./data/AIT-Wiki.txt')
document3 = text_loader.load()

In [9]:
documents = document1 + document2 + document3

In [10]:
len(documents)


8

In [11]:
documents[1]


Document(page_content='Department of Food, \nAgriculture and Bioresources\nAgribusiness Management \nAgricultural Systems and \nEngineering \nAquaculture and Aquatic \nResources Management \nFood Engineering and \nBioprocess Technology \nFood Innovation, Nutrition and \nHealth\nS T U DY  O P T I O N S  A T  A I T\nMaster’s and PhD Degree Programs\nAvailable in many fields of specializations across its \nthree Schools, covering various aspects of techno-\nlogy, social developments, and management. Sev-\neral new degree programs are designed to address \nemerging global challenges and to respond to cur-\nrent demands of the workforce. \nFlexible Degree Options\nNew alternative to traditional “On-campus Degree \nOptions”. Students can do many courses online via \nAIT’s Hybrid Modes of Instruction, with an option of \nflexible payment: course-by-course. The minimum \nenrollment is two courses per semester – ideal for those \nnot wanting to take too much time away from career, \nor those wa

Document Transformers

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [13]:
doc[1]

Document(page_content='Regional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment \nand sustainable development goals \nthrough research and capacity building\nBelt & Road Research Center \n--Research and studies in sustainability \nissues of the Belt and Road region\nInternet Education and Research \nLaboratory \n--Development, training, and education \nprograms related to internet development \nand IT topics\nAIT Artificial Intelligence\nTechnology Center\n- - R e s e a r c h  a n d  d eve l o p m e n t  t o \nincorporate AI into AIT programs and its \napplication to real-world problems \nYunus Center at AIT\n- - E nte r p r i s e  s o l u t i o n s fo r  S D G s &', metadata={'source': './drive/MyDrive/nlp-assignment-a7/data/AIT-BROCHURE.pdf', 'file_path': './drive/MyDrive/nlp-assignment-a7/data/AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (

In [14]:
len(doc)

121

#### Text Embedding Models

In [15]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


#### Vector Stores

In [16]:
# locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [17]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    # folder_path = os.path.join(vector_path, db_file_name),
    folder_path = os.path.join('./vector-store', db_file_name),
    index_name = 'nlp' #default index
)

##### retrievers

In [18]:
#calling vector from local
vector_path = './vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [19]:
#ready to use
retriever = vectordb.as_retriever()

In [20]:
retriever.get_relevant_documents("Student Exchange Program")


[Document(page_content='O P P O R T U N I T I E S\nStudent Exchange Program\nStudent Exchange Program offers an \nopportunity for AIT students to learn inside \nand outside classrooms in a partnering  \nworld-class university. Students in this program \nwill cultivate academic knowledge, transversal \nskills, and  intercultural communication, and will \nalso expand their personal and professional \nnetworks. The duration period of this program \nis one semester. Students are encouraged to \ntransfer 6-12 credits of courses taken at the \nexchange partner. Exchange scholarships are \nalso available but are competitive. \nInternship Program\nAIT offers an opportunity to all students \nenrolling in Master’s degree programs', metadata={'source': 'data/AIT-BROCHURE.pdf', 'file_path': 'data/AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Bui

In [21]:
retriever.get_relevant_documents("Outreach Centers")

[Document(page_content='Apart from academic programs in three \nAIT Schools, these Outreach Centers are \nthe connectors for AIT’s education and re-\nsearch outreach with industry and society.\nAIT Extension\n--Executive education, lifelong learning, skill \ndevelopment through short-courses, trainings, \nworkshops, and seminars\nAIT Solutions\n- - Innovative consulting/solutions in \nengineering, technology, environment, \ninfrastructure, education and management\nGeoinformatics Center \n--Consulting and capacity building in \napplication of Remote Sensing and \nGeoinformatics in agriculture, disaster risk \nmanagement, and environment\nRegional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment', metadata={'source': 'data/AIT-BROCHURE.pdf', 'file_path': 'data/AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 

#### Memory

In [22]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [23]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [24]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

#### Memory types

In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

In [27]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

#### Chain

In [28]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [31]:
print(torch.__version__)

2.2.1+cu121


In [32]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = './models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

In [33]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [36]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [37]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [38]:
query = 'Comparing both of them'
chat_history = "Human:List all faculty members of AIT:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:List all faculty members of AIT:
Follow Up Input: Comparing both of them
Standalone question:


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.


{'chat_history': 'Human:List all faculty members of AIT:',
 'question': 'Comparing both of them',
 'text': '<pad> What  are  the  faculty  members  of  AIT  and  what  are  their  roles  and  responsibilities?\n'}

In [39]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!\n   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,\n   or anything else related to AIT, I\'m here to help. Just ask away, and I\'ll provide you with all the necessary details\n   to make your experience at AIT as smooth and enjoyable as possible."\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7a239b9cb5e0>)), document_variable_name='context')

In [40]:
query = "Who are the faculty?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!
   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,
   or anything else related to AIT, I'm here to help. Just ask away, and I'll provide you with all the necessary details
   to make your experience at AIT as smooth and enjoyable as possible."
    Alumni

Faculty
AIT currently has 56 regular faculty members,[51] with 27 in the School of Environment, Resources and Development; 30 in School of Engineering and Technology; and six in the School of Management.

In the School of Environment, Resources, and Development; ten faculty are working in Food, Agriculture & Bioresources, 12 in Development and Sustainability, and 15 in Energy, Environment and Climate Change.

In the School of Engineering and Technology, 14 work in Civil and I

{'input_documents': [Document(page_content='Alumni', metadata={'source': 'data/AIT-Wiki.txt'}),
  Document(page_content='Faculty\nAIT currently has 56 regular faculty members,[51] with 27 in the School of Environment, Resources and Development; 30 in School of Engineering and Technology; and six in the School of Management.\n\nIn the School of Environment, Resources, and Development; ten faculty are working in Food, Agriculture & Bioresources, 12 in Development and Sustainability, and 15 in Energy, Environment and Climate Change.\n\nIn the School of Engineering and Technology, 14 work in Civil and Infrastructure Engineering, seven in Industrial Systems Engineering, and nine in Information and Communication Technologies.\n\nThe School of Management has six faculty members including the Dean.', metadata={'source': 'data/AIT-Wiki.txt'}),
  Document(page_content='Meet our faculty:\n\nProf. Ajit P. Annachhatre\n\nDr. Alan O’Niell\n\nDr. Amararatne Yakupitiyage\n\nMr. Andrew Macintosh\n\nPro

In [41]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!\n   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,\n   or anything else related to AIT, I\'m here to help. Just ask away, and I\'ll provide you with all the necessary details\n   to make your experience at AIT as smooth and enjoyable as possible."\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7a239b9cb5e0>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=P

#### Chatbot

In [42]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!
   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,
   or anything else related to AIT, I'm here to help. Just ask away, and I'll provide you with all the necessary details
   to make your experience at AIT as smooth and enjoyable as possible."
    Meet our faculty:

Prof. Ajit P. Annachhatre

Dr. Alan O’Niell

Dr. Amararatne Yakupitiyage

Mr. Andrew Macintosh

Prof. Anil Kumar Anal

Dr. Anish Ghimire

Dr. Atikorn Panya

Prof. ATM Nurul Amin

Dr. Attaphongse Taparugssanagorn

Dr. Avirut PuttiwongrakShow filters

Prof. Bonaventura H. W. Hadikusumo

Mr. Burin Kunatippapong

Dr. Chaiwut Gamonpilas

Dr. Chaklam Silpasuwanchai

Dr. Chantri Polprasert

Mr. Charnkurt Yaoyuenyong



Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "<pad>  I  am  an  AI  language  model  created  to  assist  users  with  information  about  Asian  Institute  of  Technology.  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  and  queries  related  to  the  school.  I  am  here  to  help  you  with  any  information  you  need  about  programs,  admission  procedures,  campus  facilities,  or  anything  else  related  to  AIT.  Just  ask  away  and  I'll  do  my  best  to  assist  you.\n",
 'source_documents': [Document(page_content='Meet our faculty:\n\nProf. Ajit P. Annachhatre\n\nDr. Alan O’Niell\n\nDr. Amararatne Yakupitiyage\n\nMr. Andrew Macintosh\n\nProf. Anil Kumar Anal\n\nDr. Anish Ghimire\n\nDr. Atikorn Panya\n\nProf. ATM Nurul Amin\n\nDr. Attaphongse Taparugssanagorn\n\nDr. Avirut PuttiwongrakShow filters\n\nProf. Bonaventura H. W. Hadikusumo\n\nMr. Burin Kunatippapong\n\nDr. Chaiwut Gamonpilas\n\nDr. Chaklam Si

In [44]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [47]:
answer['answer'].replace('<pad>', '').strip()

"I  am  an  AI  language  model  created  to  assist  users  with  information  about  Asian  Institute  of  Technology.  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  and  queries  related  to  the  school.  I  am  here  to  help  you  with  any  information  you  need  about  programs,  admission  procedures,  campus  facilities,  or  anything  else  related  to  AIT.  Just  ask  away  and  I'll  do  my  best  to  assist  you."

In [48]:
prompt_question = "List all the faculties of AIT"
answer = chain({"question":prompt_question})


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I  am  an  AI  language  model  created  to  assist  users  with  information  about  Asian  Institute  of  Technology.  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  and  queries  related  to  the  school.  I  am  here  to  help  you  with  any  information  you  need  about  programs,  admission  procedures,  campus  facilities,  or  anything  else  related  to  AIT.  Just  ask  away  and  I'll  do  my  best  to  assist  you.\n")]
Follow Up Input: List all the faculties of AIT
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.

> Finished chain.

> Finished chain.


'html>\n School  of  Environment,  Resources  and  Development\n School  of  Engineering  and  Technology\n School  of  Management'

In [49]:
answer['answer'].replace('<pad>', '').strip()

'html>\n School  of  Environment,  Resources  and  Development\n School  of  Engineering  and  Technology\n School  of  Management'

In [50]:
prompt_question = "Provide the brief history of AIT"
answer = chain({"question":prompt_question})




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I  am  an  AI  language  model  created  to  assist  users  with  information  about  Asian  Institute  of  Technology.  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  and  queries  related  to  the  school.  I  am  here  to  help  you  with  any  information  you  need  about  programs,  admission  procedures,  campus  facilities,  or  anything  else  related  to  AIT.  Just  ask  away  and  I'll  do  my  best  to  assist  you.\n"), HumanMessage(content='List all the faculties of AIT'), AIMessage(content='<pad>   html>\n School  of  Environment,  Resources  and  Development\n School  of  Engineerin

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"Welcome to the Friendly and Assistive Informational Bot for Asian Institute of Technology!
   How can I assist you today? Whether you need information about programs, admission procedures, campus facilities,
   or anything else related to AIT, I'm here to help. Just ask away, and I'll provide you with all the necessary details
   to make your experience at AIT as smooth and enjoyable as possible."
    Asian Institute of Technology
The Asian Institute of Technology (AIT), founded in 1959, is an international organization for higher education situated 40km north of Bangkok, Thailand. It specializes in engineering, advanced technologies, sustainable development, and management and planning. It aims to promote technological change and sustainable development in the Asia-Pacific region, through higher education, research, and outreach.[1]

History
Founded in 1959 as 

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.

> Finished chain.

> Finished chain.


In [53]:
answer['answer'].replace('<pad>', '').strip()

'< pad>\n The  Asian  Institute  of  Technology  (AIT)  was  founded  in  1959  as  SEATO  Graduate  School  of  Engineering.  It  was  established  to  provide  engineering  education  and  research  to  students  from  Southeast  Asia  and  the  Pacific  region.  The  school  was  initially  located  in  Bangkok,  Thailand,  but  it  later  moved  to  Thammasat  Rangsit,  a  city  located  40km  north  of  Bangkok.  The  school  was  renamed  to  Asian  Institute  of  Technology  (AIT)  in  1985.  Today,  AIT  is  an  international  organization  for  higher  education  situated  40km  north  of  Bangkok,  Thailand.  It  specializes  in  engineering,  advanced  technologies,  sustainable  development,  and  management  and  planning.  It  aims  to  promote  technological  change  and  sustainable  development  in  the  Asia-Pacific'

In [54]:
prompt_question = "How can i get admission at AIT"
answer = chain({"question":prompt_question})

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I  am  an  AI  language  model  created  to  assist  users  with  information  about  Asian  Institute  of  Technology.  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  and  queries  related  to  the  school.  I  am  here  to  help  you  with  any  information  you  need  about  programs,  admission  procedures,  campus  facilities,  or  anything  else  related  to  AIT.  Just  ask  away  and  I'll  do  my  best  to  assist  you.\n"), HumanMessage(content='List all the faculties of AIT'), AIMessage(content='<pad>   html>\n School  of  Environment,  Resources  and  Development\n School  of  Engineerin

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.

> Finished chain.

> Finished chain.


In [55]:
answer['answer'].replace('<pad>', '').strip()

'< p>Admission  to  AIT  is  open  to  students  from  all  countries  who  meet  the  eligibility  criteria  and  meet  the  academic  requirements  for  the  program  they  are  applying  for.  The  application  process  typically  involves  submitting  an  online  application,  a  personal  statement,  and  a  transcript  of  academic  records.  The  application  process  may  vary  depending  on  the  program  and  the  specific  requirements  of  the  program.\n To  apply  for  admission  to  AIT,  you  can  visit  the  official  website  of  the  program  you  are  interested  in  and  follow  the  instructions  to  submit  your  application.  You  can  also  contact  the  Admissions  Office  for  more  information  on  the  application  process  and  requirements.'